# House Buying KNN Experiment

Standalone KNN experiment extracted from `house_buying_nn_experiment.ipynb`.


In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"  # Helps avoid MKL KMeans warnings on Windows

from pathlib import Path
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    f1_score,
    balanced_accuracy_score,
    confusion_matrix,
    classification_report,
    log_loss,
    roc_auc_score,
    roc_curve,
)
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.cluster._kmeans")

sns.set_theme(style="whitegrid")
np.random.seed(42)


In [ ]:
PROJECT_DIR = Path.cwd()
if not (PROJECT_DIR / "csv").exists():
    PROJECT_DIR = PROJECT_DIR.parent

DATA_DIR = PROJECT_DIR / "csv"

train_df = pd.read_csv(DATA_DIR / "house_buy_train.csv")
cv_df = pd.read_csv(DATA_DIR / "house_buy_cv.csv")
test_df = pd.read_csv(DATA_DIR / "house_buy_test.csv")

feature_cols = [
    "buyer_income_lpa",
    "house_price_lakh",
    "loan_eligibility",
    "credit_score",
    "down_payment_percent",
    "existing_emi_lpa",
    "employment_years",
    "dependents",
    "property_location_score",
    "employment_type",
]

numeric_cols = [
    "buyer_income_lpa",
    "house_price_lakh",
    "credit_score",
    "down_payment_percent",
    "existing_emi_lpa",
    "employment_years",
    "dependents",
    "property_location_score",
]

cat_cols = ["loan_eligibility", "employment_type"]

label_to_int = {"no": 0, "neutral": 1, "yes": 2}
int_to_label = {v: k for k, v in label_to_int.items()}
class_order = np.array([0, 1, 2])
class_names = [int_to_label[i] for i in class_order]

X_train_raw = train_df[feature_cols]
y_train = train_df["can_buy"].map(label_to_int).to_numpy()
X_cv_raw = cv_df[feature_cols]
y_cv = cv_df["can_buy"].map(label_to_int).to_numpy()
X_test_raw = test_df[feature_cols]
y_test = test_df["can_buy"].map(label_to_int).to_numpy()

print("Train shape:", train_df.shape)
print("CV shape:", cv_df.shape)
print("Test shape:", test_df.shape)
print("Class distribution (train):", train_df["can_buy"].value_counts().to_dict())


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

X_train = preprocessor.fit_transform(X_train_raw)
X_cv = preprocessor.transform(X_cv_raw)
X_test = preprocessor.transform(X_test_raw)

# Convert to dense arrays so every model (including custom clustering models) can consume the same inputs.
X_train = X_train.toarray() if hasattr(X_train, "toarray") else np.asarray(X_train)
X_cv = X_cv.toarray() if hasattr(X_cv, "toarray") else np.asarray(X_cv)
X_test = X_test.toarray() if hasattr(X_test, "toarray") else np.asarray(X_test)

print("Processed feature count:", X_train.shape[1])


In [ ]:
class KMeansLabelModel:
    def __init__(self, n_clusters=7, random_state=42):
        self.n_clusters = n_clusters
        self.random_state = random_state

    def fit(self, X, y, n_classes=3):
        self.model = KMeans(
            n_clusters=self.n_clusters,
            init="k-means++",
            n_init=20,
            random_state=self.random_state,
        )
        cluster_ids = self.model.fit_predict(X)
        self.classes_ = np.arange(n_classes)
        self.cluster_proba_ = np.zeros((self.n_clusters, n_classes), dtype=float)

        for c in range(self.n_clusters):
            idx = np.where(cluster_ids == c)[0]
            if len(idx) == 0:
                self.cluster_proba_[c] = np.ones(n_classes) / n_classes
            else:
                counts = np.bincount(y[idx], minlength=n_classes).astype(float)
                self.cluster_proba_[c] = (counts + 1.0) / (counts.sum() + n_classes)
        return self

    def predict_proba(self, X):
        nearest_cluster = self.model.predict(X)
        return self.cluster_proba_[nearest_cluster]

    def predict(self, X):
        return self.predict_proba(X).argmax(axis=1)


class KMedoidsLabelModel:
    def __init__(self, n_clusters=7, random_state=42, max_iter=30):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.max_iter = max_iter

    def fit(self, X, y, n_classes=3):
        rng = np.random.default_rng(self.random_state)
        n_samples = X.shape[0]
        k = min(self.n_clusters, n_samples)

        medoid_idx = rng.choice(n_samples, size=k, replace=False)

        for _ in range(self.max_iter):
            d2 = ((X[:, None, :] - X[medoid_idx][None, :, :]) ** 2).sum(axis=2)
            assignments = d2.argmin(axis=1)

            changed = False
            new_medoid_idx = medoid_idx.copy()

            for c in range(k):
                pts = np.where(assignments == c)[0]
                if len(pts) == 0:
                    continue
                cluster_pts = X[pts]
                within_cluster_d2 = ((cluster_pts[:, None, :] - cluster_pts[None, :, :]) ** 2).sum(axis=2)
                best_local_pt = pts[within_cluster_d2.sum(axis=1).argmin()]
                if best_local_pt != medoid_idx[c]:
                    changed = True
                    new_medoid_idx[c] = best_local_pt

            medoid_idx = new_medoid_idx
            if not changed:
                break

        self.medoids_ = X[medoid_idx]
        self.classes_ = np.arange(n_classes)

        d2_final = ((X[:, None, :] - self.medoids_[None, :, :]) ** 2).sum(axis=2)
        assignments = d2_final.argmin(axis=1)

        self.cluster_proba_ = np.zeros((k, n_classes), dtype=float)
        for c in range(k):
            idx = np.where(assignments == c)[0]
            if len(idx) == 0:
                self.cluster_proba_[c] = np.ones(n_classes) / n_classes
            else:
                counts = np.bincount(y[idx], minlength=n_classes).astype(float)
                self.cluster_proba_[c] = (counts + 1.0) / (counts.sum() + n_classes)
        return self

    def predict_proba(self, X):
        d2 = ((X[:, None, :] - self.medoids_[None, :, :]) ** 2).sum(axis=2)
        nearest_medoid = d2.argmin(axis=1)
        return self.cluster_proba_[nearest_medoid]

    def predict(self, X):
        return self.predict_proba(X).argmax(axis=1)


class SeedAveragedMLP:
    def __init__(self, base_params, seeds):
        self.base_params = dict(base_params)
        self.seeds = list(seeds)
        self.models = []
        self.classes_ = np.array([0, 1, 2])

    def fit(self, X, y):
        self.models = []
        for seed in self.seeds:
            model = MLPClassifier(random_state=seed, **self.base_params)
            model.fit(X, y)
            self.models.append(model)
        return self

    def predict_proba(self, X):
        if not self.models:
            raise RuntimeError("SeedAveragedMLP must be fitted before prediction.")

        n_classes = len(self.classes_)
        avg = np.zeros((X.shape[0], n_classes), dtype=float)
        for model in self.models:
            proba = model.predict_proba(X)
            aligned = np.zeros((X.shape[0], n_classes), dtype=float)
            for col_idx, cls in enumerate(model.classes_):
                aligned[:, int(cls)] = proba[:, col_idx]
            avg += aligned
        return avg / len(self.models)


In [ ]:
def build_base_models(seed):
    return {
        "nn_small": SeedAveragedMLP(
            base_params={
                "hidden_layer_sizes": (24,),
                "activation": "relu",
                "solver": "adam",
                "alpha": 0.01,
                "learning_rate_init": 0.0012,
                "max_iter": 2200,
                "early_stopping": True,
                "validation_fraction": 0.2,
                "n_iter_no_change": 25,
            },
            seeds=[seed + 1, seed + 11, seed + 21, seed + 31, seed + 41],
        ),
        "nn_medium": MLPClassifier(
            hidden_layer_sizes=(44, 22),
            activation="relu",
            solver="adam",
            alpha=0.02,
            learning_rate_init=0.0007,
            max_iter=2600,
            early_stopping=True,
            validation_fraction=0.22,
            n_iter_no_change=20,
            random_state=seed + 3,
        ),
        "nn_deep": MLPClassifier(
            hidden_layer_sizes=(56, 28, 14),
            activation="relu",
            solver="adam",
            alpha=0.012,
            learning_rate_init=0.001,
            max_iter=2800,
            early_stopping=True,
            validation_fraction=0.2,
            n_iter_no_change=25,
            random_state=seed + 4,
        ),
        "knn": KNeighborsClassifier(n_neighbors=11, weights="distance"),
        "kmeanspp": KMeansLabelModel(n_clusters=9, random_state=seed + 5),
        "kmedoids": KMedoidsLabelModel(n_clusters=9, random_state=seed + 6, max_iter=35),
    }


def fit_model(model, X, y, n_classes):
    if isinstance(model, (KMeansLabelModel, KMedoidsLabelModel)):
        model.fit(X, y, n_classes=n_classes)
    else:
        model.fit(X, y)
    return model


def predict_proba_full(model, X, n_classes):
    proba = model.predict_proba(X)

    if proba.shape[1] == n_classes:
        return proba

    full = np.zeros((proba.shape[0], n_classes), dtype=float)
    if hasattr(model, "classes_"):
        for col_idx, cls in enumerate(model.classes_):
            full[:, int(cls)] = proba[:, col_idx]
    else:
        full[:, : proba.shape[1]] = proba
    row_sum = full.sum(axis=1, keepdims=True)
    row_sum[row_sum == 0] = 1.0
    return full / row_sum


def evaluate_split(y_true, y_pred, y_proba, n_classes):
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision_macro": precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)[0],
        "recall_macro": precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)[1],
        "f1_macro": precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)[2],
        "balanced_accuracy": balanced_accuracy_score(y_true, y_pred),
    }

    labels = np.arange(n_classes)
    y_true_bin = label_binarize(y_true, classes=labels)

    try:
        metrics["log_loss"] = log_loss(y_true, y_proba, labels=labels)
    except Exception:
        metrics["log_loss"] = np.nan

    try:
        metrics["roc_auc_macro_ovr"] = roc_auc_score(y_true_bin, y_proba, average="macro", multi_class="ovr")
    except Exception:
        metrics["roc_auc_macro_ovr"] = np.nan

    return metrics


def evaluate_model_on_splits(model, splits, n_classes):
    pred_store = {}
    proba_store = {}
    rows = []

    for split_name, (X_split, y_split) in splits.items():
        split_proba = predict_proba_full(model, X_split, n_classes=n_classes)
        split_pred = split_proba.argmax(axis=1)

        pred_store[split_name] = split_pred
        proba_store[split_name] = split_proba

        row = evaluate_split(y_split, split_pred, split_proba, n_classes=n_classes)
        row["split"] = split_name
        rows.append(row)

    return rows, pred_store, proba_store


def select_ensemble_families(base_metrics_df, top_k=2):
    cv_rank = (
        base_metrics_df[base_metrics_df["split"] == "cv"]
        .sort_values("f1_macro", ascending=False)
    )
    return cv_rank.head(top_k)["model"].tolist()


In [ ]:
splits = {
    "train": (X_train, y_train),
    "cv": (X_cv, y_cv),
    "test": (X_test, y_test),
}

n_classes = len(class_order)
model = KNeighborsClassifier(n_neighbors=11, weights="distance")
fit_model(model, X_train, y_train, n_classes=n_classes)

rows, pred_store, _ = evaluate_model_on_splits(model, splits, n_classes=n_classes)
knn_metrics_df = pd.DataFrame(rows)

metric_cols = [
    "accuracy",
    "precision_macro",
    "recall_macro",
    "f1_macro",
    "balanced_accuracy",
    "log_loss",
    "roc_auc_macro_ovr",
]

display(knn_metrics_df.set_index("split")[metric_cols])
print("\nTest classification report (KNN):")
print(classification_report(y_test, pred_store["test"], target_names=class_names))
